In [5]:
import pandas as pd
df = pd.read_csv("med_cost_predict.csv")
df.head(10)

,age,gender,bmi,smoker,diabetes,hypertension,heart_disease,asthma,physical_activity_level,daily_steps,sleep_hours,stress_level,doctor_visits_per_year,hospital_admissions,medication_count,insurance_type,insurance_coverage_pct,city_type,previous_year_cost,annual_medical_cost
0,69,Male,29.4,No,1,0,0,0,Medium,14825,4.4,8,1,0,4,Private,80,Semi-Urban,10885,2645.50
1,32,Female,22.9,No,1,0,0,0,Medium,3620,6.0,7,4,3,0,Government,64,Semi-Urban,18722,10959.70
2,89,Male,25.7,No,0,0,0,0,High,10578,4.5,7,2,0,3,NaN,0,Urban,4196,8409.80
3,78,Male,31.9,Yes,0,1,0,0,Low,6226,8.6,9,6,1,7,Government,70,Urban,11128,7996.62
4,38,Male,27.7,No,0,0,0,0,High,6253,5.7,3,6,0,6,Private,77,Urban,15110,3202.52
5,41,Male,25.7,No,0,0,0,0,Medium,14720,7.7,10,5,2,6,Private,89,Urban,18459,3072.16
6,20,Female,23.0,Yes,0,0,0,0,High,14762,4.4,8,5,0,4,Government,71,Rural,10047,4032.19
7,39,Female,22.8,No,0,0,0,0,Low,9726,4.6,6,3,1,2,Government,66,Rural,13775,5302.81
8,70,Male,21.7,Yes,0,0,0,0,Low,10800,7.4,8,8,0,2,Government,54,Rural,8289,6411.34
9,19,Male,23.3,No,0,0,0,0,Medium,3398,8.2,4,5,1,4,Government,50,Urban,16291,8983.15


In [6]:
df['hospital_admissions'] = df['hospital_admissions'].astype(str).str.strip()
df['hospital_admissions'] = pd.to_numeric(df['hospital_admissions'], errors='coerce')
df['hospital_admissions'].head()

0    0
1    3
2    0
3    1
4    0
Name: hospital_admissions, dtype: int64

In [8]:
from sklearn.preprocessing import StandardScaler
from  sklearn.model_selection import train_test_split
X = df.drop('annual_medical_cost', axis=1) # Features
y = df['annual_medical_cost']              # Target

# Split the data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Columns requiring scaling (StandardScaler)
numerical_features = ['age', 'bmi', 'sleep_hours', 'hospital_admissions']

# Columns requiring Label/Ordinal Encoding
ordinal_features = ['physical_activity_level']

# Columns requiring One-Hot Encoding
nominal_features = ['gender', 'smoker', 'insurance_type']

# Binary columns (already 0/1) that can be passed through
binary_features = ['diabetes']

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Replace NaN with median
    ('scaler', StandardScaler())                    # Apply Z-score scaling
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Replace NaN with mode
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # One-Hot Encoding
])


def map_ordinal_features(df):
    mapping = {'Low': 1, 'Medium': 2, 'High': 3}
    df['physical_activity_level'] = df['physical_activity_level'].map(mapping)
    return df

X_train = map_ordinal_features(X_train.copy())
X_test = map_ordinal_features(X_test.copy())

# Adjust numerical features to include the newly mapped ordinal column
numerical_features.append('physical_activity_level')

In [16]:
from sklearn.preprocessing import StandardScaler

# Fit and transform the training data
X_train_processed = preprocessor.fit_transform(X_train)

# Transform the testing data (DO NOT FIT on test data)
X_test_processed = preprocessor.transform(X_test)

# To see the result as a DataFrame (optional, but helpful)
feature_names = preprocessor.get_feature_names_out()
X_train_processed_df = pd.DataFrame(X_train_processed, columns=feature_names)

print("\n--- Processed Training Data (First 3 rows) ---")
print(X_train_processed_df.head(3))

NameError: name 'preprocessor' is not defined